We will define an environment that will give the agent random rewards for a limited number of steps, regardless of the agent's actions. This scenario is not very useful, but it will allow us to focus on specific methods in both the environment and agent classes.

In [1]:
import random 
from typing import List

class Environment:
    def __init__(self):
        self.steps_left = 10
    
    def get_observation(self) -> List[float]:
        return [0.0, 0.0, 0.0]
    
    def get_actions(self) -> List[int]:
        return [0, 1]
    
    def is_done(self) -> bool:
        return self.steps_left == 0
    
    def action(self, action) -> float:
        if self.is_done():
            raise Exception('Game is over.')
        self.steps_left -= 1
        return random.random()

The `get_observation()` method is supposed to return the current environment's observation to the agent. It is usually implemented as some function of the internal state of the environment. 

The `get_actions()` method allows the agent to query the set of actions it can execute. Normally, the set of actions that the agent can execute does not change over time, but some actions can become impossible in different states. In this simplistic example, there are only two actions that the agent can carry out, which are encoded as 0 and 1.

The `action()` method is the central piece in the environment's functionality. It does two things - handles the agent's action and returns the reward for this action. In our example, the reward is random and its action is discarded. 

In [2]:
class Agent:
    def __init__(self):
        self.total_reward = 0.0
        
    def step(self, env):
        current_obs = env.get_observation()
        actions = env.get_actions()
        reward = env.action(random.choice(actions))
        self.total_reward += reward

The `step` function accepts the environment instance as an argument and allows the agent to perform the following actions:

* Observe the environment.
* Make a decision about the action to take based on the observations. 
* Submit the action to the environment.
* Get the reward for the current step.

Our agent is dull and ignores the observations obtained during the decision-making process about which action to take. Instead, every action is selected randomly.

In [3]:
if __name__ == '__main__':
    env = Environment()
    agent = Agent()
    
    while not env.is_done():
        agent.step(env)
        
    print(f'Total Reward: {agent.total_reward:.4f}')

Total Reward: 3.2498
